In [26]:
import torch # PyTorch
from torchvision import datasets, transforms # Datasets y Transformaciones (Para la parte de datos)
from torch.utils.data import DataLoader # Para cargar los datos en la red
from torch import nn, optim # Para crear la red y el optimizador
import torch.nn.functional as F # Funciones de activación y pérdida

In [27]:
# Configuración
BATCH_SIZE = 64
DATA_DIR = "./data"

### Preparación de datos
El primer paso es preparar los datos que vamos a utilizar para entrenar la red. Esto incluye cargar los datos, aplicar transformaciones necesarias (como normalización, aumento de datos, etc.) y dividirlos en conjuntos de entrenamiento y test. Suele ser comun dividir los datos en un 80% para entrenamiento y un 20% para test.

In [28]:
# Funciones útiles
def get_transform():
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

def get_dataset(train=True, transform=None):
    return datasets.MNIST(
        root=DATA_DIR,
        train=train,
        download=True,
        transform=transform
    )

def get_dataloader(dataset, batch_size=BATCH_SIZE, shuffle=True):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle
    )

In [29]:
# Preparar los datos
def load_mnist(batch_size=BATCH_SIZE):
    transform = get_transform()
    train_dataset = get_dataset(train=True, transform=transform)
    test_dataset = get_dataset(train=False, transform=transform)

    train_loader = get_dataloader(train_dataset, batch_size, shuffle=True)
    test_loader = get_dataloader(test_dataset, batch_size, shuffle=False)

    return train_loader, test_loader, train_dataset

#### Creación de la red

A continuación, definimos la arquitectura de la red neuronal. Esto implica decidir el número de capas, el tipo de capas (convolucionales, lineales, etc.), las funciones de activación y cómo se conectan entre sí.

Para definir una red neuronal en PyTorch, se crea una clase que hereda de `nn.Module`. Dentro de esta clase, se definen las capas de la red en el método `__init__` y la lógica de cómo los datos fluyen a través de estas capas en el método `forward`. Aquí hay un ejemplo simple de una red neuronal con dos capas completamente conectadas (fully connected):

Las fully connected layers (o capas densas) son aquellas en las que cada neurona de una capa está conectada a todas las neuronas de la capa siguiente. Esto permite que la red aprenda representaciones complejas de los datos, ya que cada neurona puede combinar información de todas las neuronas anteriores.

nn.Linear(in_features, out_features) es una capa lineal que aplica una transformación lineal a los datos de entrada. Aquí, in_features es el número de características de entrada (por ejemplo, el tamaño del vector de entrada) y out_features es el número de características de salida (por ejemplo, el número de neuronas en la capa).

In [30]:
from torch import nn

class ModularNN(nn.Module):
    def __init__(self, input_size=28*28, hidden_layers=[128, 64], output_size=10, activation=nn.ReLU):
        """
        input_size: tamaño de entrada (por defecto 28*28 para imágenes MNIST)
        hidden_layers: lista con el número de neuronas por cada capa oculta
        output_size: número de clases de salida
        activation: clase de función de activación (nn.ReLU, nn.Tanh, nn.Sigmoid, etc.)
        """
        super(ModularNN, self).__init__()
        
        self.layers = nn.ModuleList()
        prev_size = input_size
        
        # Crear las capas ocultas
        for hidden_size in hidden_layers:
            self.layers.append(nn.Linear(prev_size, hidden_size))
            self.layers.append(activation())  # Función de activación
            prev_size = hidden_size
        
        # Capa de salida
        self.layers.append(nn.Linear(prev_size, output_size))
        self.layers.append(nn.Softmax(dim=1))
        
        # Convertir la lista de capas en un Sequential
        self.network = nn.Sequential(*self.layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Aplanar la entrada
        return self.network(x)


```python 
# Red con 2 capas ocultas de 128 y 64 neuronas
model = ModularNN(hidden_layers=[128, 64])

# Red con 3 capas ocultas de 256, 128 y 64 neuronas
model2 = ModularNN(hidden_layers=[256, 128, 64], activation=nn.Tanh)
```

In [31]:
from torchsummary import summary

def summary(model, input_size):
    sum = summary(model, input_size)
    print(sum)

### Optimizador

La función de pérdida mide qué tan bien está funcionando la red. El optimizador ajusta los pesos de la red para minimizar la pérdida.

In [32]:
def get_loss_function(loss_name='MSELoss', **kwargs):
    """
    Devuelve la función de pérdida configurada.
    
    loss_name: 'MSELoss', 'CrossEntropyLoss', 'L1Loss', etc.
    kwargs: otros parámetros específicos de la función de pérdida
    """
    if loss_name == 'MSELoss':
        return nn.MSELoss(**kwargs)
    elif loss_name == 'CrossEntropyLoss':
        return nn.CrossEntropyLoss(**kwargs)
    elif loss_name == 'L1Loss':
        return nn.L1Loss(**kwargs)
    else:
        raise ValueError(f"Función de pérdida '{loss_name}' no soportada")

def get_optimizer(model, optimizer_name='SGD', lr=0.01, **kwargs):
    if optimizer_name == 'SGD':
        return optim.SGD(model.parameters(), lr=lr, **kwargs)
    elif optimizer_name == 'Adam':
        return optim.Adam(model.parameters(), lr=lr, **kwargs)
    elif optimizer_name == 'RMSprop':
        return optim.RMSprop(model.parameters(), lr=lr, **kwargs)
    else:
        raise ValueError(f"Optimizador '{optimizer_name}' no soportado")

```python
# Loss function
criterion = get_loss_function(loss_name='CrossEntropyLoss')

# Optimizer
optimizer = get_optimizer(model, optimizer_name='Adam', lr=0.001)

```

### Entrenamiento de la red

En este paso, alimentamos los datos de entrenamiento a la red, calculamos la pérdida, y actualizamos los pesos utilizando el optimizador. Este proceso se repite durante varias épocas.

In [33]:
import torch
import torch.nn.functional as F

def train_model(model, train_loader, criterion, optimizer, device=torch.device("cpu"), epochs=5, one_hot=False, num_classes=10, verbose=True):
    """
    Función modular para entrenar un modelo PyTorch.
    
    model: nn.Module
    train_loader: DataLoader con los datos de entrenamiento
    criterion: función de pérdida
    optimizer: optimizador
    device: 'cpu' o 'cuda'
    epochs: número de épocas de entrenamiento
    one_hot: si True convierte las etiquetas a one-hot (útil para MSELoss)
    num_classes: número de clases (para one-hot)
    verbose: si True imprime la pérdida por época
    """
    
    model = model.to(device)
    
    for epoch in range(epochs):
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            
            if one_hot:
                labels = F.one_hot(labels, num_classes=num_classes).float()
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        if verbose:
            avg_loss = running_loss / len(train_loader)
            print(f'[Epoch {epoch + 1}] loss: {avg_loss:.3f}')


### Evaluación de la red

Para evaluar el rendimiento de la red, se utiliza el conjunto de test. Esto implica pasar los datos de test a través de la red y comparar las predicciones con las etiquetas reales para calcular métricas como la precisión (accuracy). El accuracy es la proporción de predicciones correctas sobre el total de predicciones realizadas. Se calcula como:
$$
\text{Accuracy} = \frac{\text{Número de predicciones correctas}}{\text{Número total de predicciones}} \times 100\%
$$

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate(model, test_loader):
    model.eval()  # Poner el modelo en modo evaluación
    correct = 0
    total = test_loader.dataset.__len__()  # Total de muestras en el conjunto de test
    print(f'Total de muestras en el conjunto de test: {total}')
    with torch.no_grad():  # No calcular gradientes
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)  # Mover datos al dispositivo
            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Obtener las predicciones
            correct += (predicted == labels).sum().item()  # Actualizar el contador de aciertos
    accuracy = 100 * correct / total if total > 0 else 0
    print(f'Accuracy: {accuracy:.2f}%')

# Ejercicio

In [37]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
experiments = [
    {"hidden_layers": [128], "activation": nn.ReLU, "optimizer": "SGD", "lr": 0.01, "batch_size": 32},
    {"hidden_layers": [256, 128], "activation": nn.ReLU, "optimizer": "Adam", "lr": 0.001, "batch_size": 64},
    {"hidden_layers": [128, 64, 32], "activation": nn.Tanh, "optimizer": "RMSprop", "lr": 0.0005, "batch_size": 128},
    {"hidden_layers": [512, 256], "activation": nn.Sigmoid, "optimizer": "SGD", "lr": 0.1, "batch_size": 32},
]

for exp in experiments:
    print(f"\n=== Experimento: {exp} ===")
    train_loader, test_loader, _ = load_mnist(batch_size=exp["batch_size"])
    
    model = ModularNN(hidden_layers=exp["hidden_layers"], activation=exp["activation"])
    optimizer = get_optimizer(model, optimizer_name=exp["optimizer"], lr=exp["lr"])
    criterion = get_loss_function('MSELoss')  # Puedes probar CrossEntropyLoss también
    
    train_model(model, train_loader, criterion, optimizer, device=DEVICE, epochs=5, one_hot=True)
    evaluate(model, test_loader)



=== Experimento: {'batch_size': 64, 'hidden_layers': [256, 128], 'activation': <class 'torch.nn.modules.activation.ReLU'>, 'loss': 'MSELoss', 'optimizer': 'Adam', 'lr': 0.001} ===
[Epoch 1] loss: 0.016
[Epoch 2] loss: 0.008
[Epoch 3] loss: 0.006
[Epoch 4] loss: 0.005
[Epoch 5] loss: 0.005
Total de muestras en el conjunto de test: 10000
Accuracy: 96.76%

=== Experimento: {'batch_size': 64, 'hidden_layers': [512, 256], 'activation': <class 'torch.nn.modules.activation.ReLU'>, 'loss': 'MSELoss', 'optimizer': 'Adam', 'lr': 0.001} ===
[Epoch 1] loss: 0.015
[Epoch 2] loss: 0.008
[Epoch 3] loss: 0.006
[Epoch 4] loss: 0.005
[Epoch 5] loss: 0.004
Total de muestras en el conjunto de test: 10000
Accuracy: 96.49%

=== Experimento: {'batch_size': 64, 'hidden_layers': [256, 128], 'activation': <class 'torch.nn.modules.activation.ReLU'>, 'loss': 'MSELoss', 'optimizer': 'Adam', 'lr': 0.001} ===
[Epoch 1] loss: 0.016
[Epoch 2] loss: 0.008
[Epoch 3] loss: 0.006
[Epoch 4] loss: 0.005
[Epoch 5] loss: 0.0

KeyboardInterrupt: 